Add project root to path

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Load raw data

In [ ]:
from astropy.io import fits
import pandas as pd
from astropy.wcs import WCS

from utils.data.segmentmap import create_from_files
from utils import filename

name = 'dev_s'
segmentmap = create_from_files(name)
hi_data = fits.getdata(filename.data.sky(name)).T
header = fits.getheader(filename.data.sky(name))
df = pd.read_csv(filename.data.true(name), sep=' ')

Power transform

In [ ]:
import numpy as np
a = 100
for i in range(hi_data.shape[-1]):
    hi_data[:, :, i] = (hi_data[:, :, i] - hi_data[:, :, i].min()) / (
            np.percentile(hi_data[:, :, i], 99.9) - hi_data[:, :, i].min())
    hi_data[:, :, i] = (np.power(a, hi_data[:, :, i]) - 1) / a

Create dataset objects

In [ ]:
from utils.data.generating import create_datacube_set_dict
import numpy as np

cube_dim = np.array([32,32,32])
empty_cube_dim = np.array([64,64,64])
dataset = create_datacube_set_dict(df, hi_data, segmentmap, WCS(header), header, .5, cube_dim, empty_cube_dim)

Create dataloader, split to train & test

In [ ]:
from utils.data import splitting

train, test = splitting.splitted_loaders(dataset, .8)

Load pretrained 2D model

In [ ]:
import segmentation_models_pytorch as smp

model = smp.Unet(encoder_name='resnet18', in_channels=1, classes=1)

Convert pretrained 2D model to 3D

In [ ]:
import torch
from models.convert2Dto3D import Conv3dConverter

Conv3dConverter(model, -1, torch.ones(1, 1, 32, 32, 32))

Create Lightning objects

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_toolbelt.losses import BinaryLovaszLoss
from pytorch_lightning.loggers import TensorBoardLogger
from datetime import datetime

from models.segmentation import Segmenter

version = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
logger = TensorBoardLogger("tb_logs", name="segmenter", version=version)
segmenter = Segmenter(model, BinaryLovaszLoss(), train, test, logger)

trainer = pl.Trainer(max_epochs=500, gpus=1, logger=logger)

Train!

In [ ]:
trainer.fit(segmenter)